# [Module 4.2.2] inference on custem tf serving image

이 노트북은 아래와 같은 작업을 진행 합니다.
- Custom Inference Docker Image를 사용하여, SageMaker Model을 생성
- 위의 SageMaker Model를 통해서 Deploy 하여 Endpoint를 생성 합니다.
- Endpoint에 Predictor를 생성 합니다.
- 추론을 하여 상위 5개의 높은 스코어를 가진 것을 이모팀콘으로 변경하여 5개 추천 합니다.

---
이 노트북은 약 10 분 정도 시간이 소요 됩니다.

In [22]:
import boto3
import sagemaker
import pandas as pd
import os

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

In [23]:
%store -r 

In [24]:
# training_job_name = 'bert2tweet-2020-07-08-07-58-27-895'
print(training_job_name)

bert2tweet-2020-08-19-05-23-18-129


In [25]:
inference_image = ecr_infer_custom_image_tf_serving_20_cpu
print("inference_image: ", inference_image)

inference_image:  343441690612.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-tensorflow-serving:2.0.0-cpu


## Custome TFS Docker Image 및 Inference code 로 모델 생성

In [26]:
from sagemaker.tensorflow.serving import Model

model = Model(model_data='s3://{}/{}/output/model.tar.gz'.format(bucket, training_job_name),
              role=role,
              entry_point='inference.py',
              image = inference_image
             ) 

In [27]:
print(model.image)
print(model.serving_image_uri)
print(model.model_data)
print(model.name)
print(model.FRAMEWORK_NAME)


343441690612.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-tensorflow-serving:2.0.0-cpu
<bound method Model.serving_image_uri of <sagemaker.tensorflow.serving.Model object at 0x7fba6721a2e8>>
s3://sagemaker-ap-northeast-2-343441690612/bert2tweet-2020-08-19-05-23-18-129/output/model.tar.gz
None
tensorflow-serving


## 엔드포인트 생성

In [28]:
%%time

import time
endpont_name    = f'custom-inference-image-endpoint-{time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())}'

instance_type='ml.m4.xlarge'
deployed_model = model.deploy(
                             endpoint_name= endpont_name,
                             initial_instance_count = 1,
                             instance_type = instance_type,
                             wait=True)


-------------!CPU times: user 39.5 s, sys: 7.05 s, total: 46.6 s
Wall time: 7min 17s


## Predictor Creation on the Endpoint

In [29]:
# tweet_bert_endpoint_name = 'train_text, train_label, test_text, test_label = tweet_data.split_train_test_data(texts, labels, 0.9)
tweet_bert_endpoint_name = deployed_model.endpoint
print(tweet_bert_endpoint_name)

custom-inference-image-endpoint-2020-08-19-05-54-44


In [30]:
import json
from sagemaker.tensorflow.serving import Predictor

predictor = Predictor(endpoint_name = tweet_bert_endpoint_name,
                      sagemaker_session = sess,
                      content_type = 'application/json',
                     )

## Inference 실행

In [31]:
from TweetUtil import TweetUtil

tweet_util = TweetUtil()
tweet_util.load_emoji_data('emoji_to_idx.pickle')
emoji = tweet_util.get_emo_class_label(3)
print(emoji)

emoji_to_idx is loaded
😂


In [32]:
test_file_path = 'data/test/tweet_file_test.csv'
test_df = pd.read_csv(test_file_path)
test_file_path = 'data/test/tweet_file_test.csv'
test_df = pd.read_csv(test_file_path)
sample_df = test_df.sample(10)
sample_df

,TWEET,LABEL
7597,u did it first,8
7022,bought beyonce tickets with my rent money now...,4
2092,you unfollow me but you still gonna look at m...,9
4469,lol i know i'm getting me a pair they're bett...,3
1992,great now more lincoln commercials,6
8232,you have to be on your period to not want to ...,8
860,aint all bad had a free nandos,4
1495,danny fuck off i didn't know it until someone...,8
8987,i'm really bored as shit,8
5989,my mood is all over the place today,8


In [33]:
def show_top_N_label(score_list, topN):

    import numpy as np

    top_n_idx = np.argsort(score_list)[-topN:]
    top_n_values = [score_list[i] for i in top_n_idx]
    
    top_n_idx_list = top_n_idx.tolist()
    top_n_idx_list.reverse()
    top_n_values = [score_list[i] for i in top_n_idx_list]    
    
    return top_n_idx_list



## Top 5 이모티콘 추천

In [34]:
import tensorflow as tf
import json

columns = ['TWEET', 'LABEL']
topN = 5
for tweet, label in zip(sample_df.TWEET.values, sample_df.LABEL.values):
    # print("label: {}, tweet: {}".format(label, tweet))
    
    reviews = [tweet]
    
#     print("reviews: \n", reviews)



    predicted_classes = predictor.predict(reviews)[0]
#    predicted_classes = predictor.predict(reviews)    
    predicted_classes = show_top_N_label(predicted_classes, topN)

    print('-------------------------------------------')        
    print('tweet: {} \nGround_truth- {}:{}\n '.format(
        tweet,
        label, 
        tweet_util.get_emo_class_label(label))
         )    
    

    print('Prediction: {},{},{},{},{},{},{},{},{},{} \n '.format(
        predicted_classes[0], 
        tweet_util.get_emo_class_label(predicted_classes[0]),
        predicted_classes[1], 
        tweet_util.get_emo_class_label(predicted_classes[1]),
        predicted_classes[2], 
        tweet_util.get_emo_class_label(predicted_classes[2]),                                       
        predicted_classes[3], 
        tweet_util.get_emo_class_label(predicted_classes[3]),                                      
        predicted_classes[4], 
        tweet_util.get_emo_class_label(predicted_classes[4]),                                      
        
        ))    
        




-------------------------------------------
tweet:  u did it first 
Ground_truth- 8:🙄
 
Prediction: 9,🤔,3,😂,4,😊,2,🔥,7,😭 
 
-------------------------------------------
tweet:  bought beyonce tickets with my rent money now i gotta be out by midnight tomorrow 
Ground_truth- 4:😊
 
Prediction: 6,😩,8,🙄,4,😊,7,😭,9,🤔 
 
-------------------------------------------
tweet:  you unfollow me but you still gonna look at my stuff how that work sway  
Ground_truth- 9:🤔
 
Prediction: 9,🤔,8,🙄,7,😭,4,😊,3,😂 
 
-------------------------------------------
tweet:  lol i know i'm getting me a pair they're better than the regular ones to me 
Ground_truth- 3:😂
 
Prediction: 3,😂,8,🙄,7,😭,4,😊,6,😩 
 
-------------------------------------------
tweet:  great now more lincoln commercials  
Ground_truth- 6:😩
 
Prediction: 2,🔥,3,😂,9,🤔,6,😩,5,😍 
 
-------------------------------------------
tweet:  you have to be on your period to not want to deal with people i guess depression isn't a real thing then https  
Ground_truth-

In [35]:
%store tweet_bert_endpoint_name

Stored 'tweet_bert_endpoint_name' (str)
